In [17]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_0 = pd.read_parquet("train_0.parquet")

# Delete target 3 to 8, feature 00 - 04, 21, 26, 27, 31
df_0 = df_0.drop(columns=[f"target_{i}" for i in range(3, 9)] + [f"feature_{i:02.0f}" for i in [0, 1, 2, 3, 4, 21, 26, 27, 31]])

In [20]:
feature_nos = list(range(78)) - [0, 1, 2, 3, 4, 21, 26, 27, 31]
features = [f"feature_{i:02.0f}" for i in list(range(78))-[0, 1, 2, 3, 4, 21, 26, 27, 31]]

df_0

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [19]:
# count the number of missing values in each column
missing_ratio = df_0.isnull().sum()/len(df_0)

for i in range(0, 78):
    if missing_ratio[f"feature_{i:02.0f}"] == 1.0:
        print(i, missing_ratio[f"feature_{i:02.0f}"])

0 1.0
1 1.0
2 1.0
3 1.0
4 1.0
21 1.0
26 1.0
27 1.0
31 1.0
